In [32]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
data_folder_path = 'C:\\Users\\goodluck\\Desktop\\DB'
raw_data_path =  "C:\\Users\\goodluck\\Desktop\\DB\\raw_data"
storage_path =  "C:\\Users\\goodluck\\Desktop\\DB\\database_storage"
daily_path =  "C:\\Users\\goodluck\\Desktop\\DB\\daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")
# re-open
with open(DCBS_path, 'rb') as f:  
    DCBS = pickle.load(f)
# re-open
with open(DTBS_path, 'rb') as f:  
    DTBS = pickle.load(f)

In [33]:
def select_condition(codes, dt, fcd):
    satisfy = []
    if fcd[0] == 'lp':
        lm = calculate_time(dt, 'p', 20)
        
        for code in codes:
            if DCBS[lm[0]].__contains__(code):
                if low_prc(code, dt, lm, fcd[1]):
                    satisfy.append(code)
        
    else:
        for code in codes:
            if DCBS[dt][code][fcd[0]] <= fcd[2] and  DCBS[dt][code][fcd[0]] >= fcd[1]:
                satisfy.append(code)
                    
    return satisfy


def takeSecond(elem):
    return elem[1]

def rank(codes, dt, idx, num, d):
    rk = []
    if num == -1:
        num = len(codes)
    for code in codes:
        if idx == 'lp':
            if not np.isnan(DCBS[dt][code]['dp']):
                lm = calculate_time(dt, 'p', 20)
                if DCBS[lm[0]].__contains__(code):
                    prc = get_prcs(code, lm)
                    lp = absolute_low(prc, 0)
                    rk.append((code,  DCBS[dt][code]['dp']/lp))
            
            
        elif idx != 'dl':
            if not np.isnan( DCBS[dt][code][idx]):
                rk.append((code,  DCBS[dt][code][idx]))
        else:
            if not np.isnan( DCBS[dt][code][idx]):
                if  DCBS[dt][code][idx] != 999:
                    rk.append((code,  DCBS[dt][code][idx]))
    if num >= len(rk):
        num = len(rk)
        
     
#     reverse = True 降序
    if d == 'd':
        rk.sort(key=takeSecond, reverse = True)
    else:
        rk.sort(key=takeSecond, reverse = False)

    if idx == 'lp':
        lp = [i for i in rk if i[1] == 1.0]
        ot = []
        if num - len(lp) > 0:
            ot = rk[len(lp):num]
        
        low = lp + ot
        
    else:    
        low = rk[0:num]
#     print("In",dt , len(low), "of", len(codes),  "is selected")   

#     print(low)
    
    
    return [i[0] for i in low]

def get_prcs(code, dts):
    prc = []
    
    for dt in dts:
        prc.append(DCBS[dt][code]['dp'])
    prc.sort()
    return prc
    
def relative_low(lst, pos):
    return np.quantile(a=lst, q = pos)
    
def absolute_low(lst, pos):
    return min(lst) + pos * (max(lst) - min(lst))

def low_prc(code, dt, lm, pos):
    prc = get_prcs(code, lm)
#     lp = min([relative_low(prc, pos), absolute_low(prc, pos)])
    lp = absolute_low(prc, pos)
    tp = DCBS[dt][code]['dp']
#     if (tp <= lp):
#         print(code, tp, lp)
#     if tp <= lp:
#         print(tp, lp)
    return tp <= lp

def calculate_time(dt, d, l):
    time = []
    days = list(DCBS.keys())
    if d == 'p':
        for i in range(len(days)):
            if days[i] == dt:
                break
                
    return days[i-l+1:i+1]



In [34]:
tdd =datetime.today()
td = tdd.strftime('%Y-%m-%d')
DCBS.keys()


dict_keys(['2023-04-27', '2023-04-28', '2023-05-04', '2023-05-05', '2023-05-08', '2023-05-09', '2023-05-10', '2023-05-11', '2023-05-12', '2023-05-15', '2023-05-16', '2023-05-17', '2023-05-18', '2023-05-19', '2023-05-22', '2023-05-23', '2023-05-24', '2023-05-25', '2023-05-26', '2023-05-29', '2023-05-30', '2023-05-31', '2023-06-01', '2023-06-02', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-19', '2023-06-20', '2023-06-21', '2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30', '2023-07-03', '2023-07-04', '2023-07-05'])

In [38]:
select = list(DCBS[td].keys())

# print(select)

# flt_cd = [['dp', 100, 130]]

# flt_cd = [['dp', 100, 130], ['bl',0, 3]]

# flt_cd = [['dp', 100, 130], ['bl',0, 3]]

# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]









# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]


# 
# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', 0.5, 9999]]

# flt_cd = [['dp', 100, 130], ['bl',0, 3]]
# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', 0.5, 9999]]
# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]

# flt_cd = [['lp', 0.5], ['dp', 100, 130], ['bl',0, 3], ['cpr', -999, 50]]






  

# 低价
flt_cd = [['lp', 0.5], ['dp', 100, 130], ['bl',0, 3], ['cpr', -999, 50]]

# 双低5叠加/区间双低3叠加
flt_cd = [['dp',100,130],['ytm', -1, 999], ['bl', 0, 7], ['trt',2,10]]

# 高ytm
flt_cd = [['dp',100,130],['ytm', 0.5, 999], ['bl', 0, 7], ['trt',2,10]]

# 双低5小市值
flt_cd = [['dp', 100, 130], ['bl',0, 3]]  




for fcd in flt_cd:

    select = select_condition(select, td, fcd)














# 低价
rk_cd = [['lp', 30, 'a']]



# 高ytm
rk_cd = [['ytm', 20, 'd']]


# 双低
rk_cd = [['dl', 30, 'a']]


codes = rank(select, td, rk_cd[0][0], rk_cd[0][1], rk_cd[0][2])

In [39]:
for r in codes:
    print(r, DCBS[td][r]['cn'], DCBS[td][r]['sc'],  DCBS[td][r]['sn'], "收盘价：", DCBS[td][r]['dp'],  "转股溢价率：", DCBS[td][r]['cpr'], "到期收益率：", DCBS[td][r]['ytm'],"换手率：", DCBS[td][r]['trt'], '市值',  DCBS[td][r]['bl'], '货币资金',  round(DTBS['F'][DTBS['B'][r]['sc']]['2023-03']['hb'],2), '双低', DCBS[td][r]['dl'])

113595.SH 花王转债 603007.SH ST花王 收盘价： 111.288 转股溢价率： 3.01 到期收益率： 3.71 换手率： 15.94 市值 2.707 货币资金 0.18 双低 114.298
123014.SZ 凯发转债 300407.SZ 凯发电气 收盘价： 117.408 转股溢价率： 3.07 到期收益率： -161.14 换手率： 13.93 市值 1.151 货币资金 6.75 双低 120.478
123002.SZ 国祯转债 300388.SZ 节能国祯 收盘价： 108.5 转股溢价率： 30.52 到期收益率： -5.92 换手率： 1.15 市值 2.521 货币资金 12.77 双低 139.02
128021.SZ 兄弟转债 002562.SZ 兄弟科技 收盘价： 110.736 转股溢价率： 32.0 到期收益率： -10.69 换手率： 1.52 市值 2.651 货币资金 3.97 双低 142.736
128033.SZ 迪龙转债 002658.SZ 雪迪龙 收盘价： 126.697 转股溢价率： 23.25 到期收益率： -34.07 换手率： 1.54 市值 2.972 货币资金 5.56 双低 149.947
113546.SH 迪贝转债 603320.SH 迪贝电气 收盘价： 128.333 转股溢价率： 22.78 到期收益率： -3.43 换手率： 3.33 市值 2.297 货币资金 3.63 双低 151.113
110070.SH 凌钢转债 600231.SH 凌钢股份 收盘价： 117.861 转股溢价率： 36.27 到期收益率： -0.71 换手率： 0.88 市值 2.17 货币资金 36.2 双低 154.131
123166.SZ 蒙泰转债 300876.SZ 蒙泰高新 收盘价： 127.47 转股溢价率： 27.87 到期收益率： -0.92 换手率： 2.82 市值 3.0 货币资金 2.09 双低 155.34
128071.SZ 合兴转债 002228.SZ 合兴包装 收盘价： 115.3 转股溢价率： 42.45 到期收益率： -0.84 换手率： 0.7 市值 2.999 货币资金 6.19 双低 157.75
113565.SH 宏辉转债 603336.SH 宏辉果蔬

In [40]:
td = DTBS['D']['day'][-1]
result = []
for cd in DTBS['A'].keys():
    if DTBS['A'][cd][td]['ia'] == 1:
        result.append(DTBS['A'][cd][td]['dp'] )
np.median(result)        

122.87049999999999